<a href="https://colab.research.google.com/github/Adilbenchadli/Hellow-world/blob/main/stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nouveau

Separation

In [12]:
# @title
%%writefile analysis.py

import pandas as pd
import numpy as np

def process_stock_data(df: pd.DataFrame):
    """
    Analyzes stock data from a DataFrame to calculate reorder quantities.
    """
    # Nettoyage initial des noms de colonnes
    df.columns = df.columns.str.strip().str.replace('\ufeff', '', regex=True)

    # Identification des colonnes de sorties
    output_cols = [col for col in df.columns if 'Somme de Sorties' in col]

    if not output_cols:
        raise ValueError("Aucune colonne contenant 'Somme de Sorties' n'a été trouvée.")

    # Conversion et nettoyage des données
    df[output_cols] = df[output_cols].fillna(0)
    df['Stock Phy'] = pd.to_numeric(df['Stock Phy'], errors='coerce').fillna(0)

    # Calculs de consommation
    total_output = df[output_cols].sum(axis=1)
    num_months = len(output_cols)
    average_monthly_output = total_output / num_months if num_months > 0 else 0

    df['conso-Moy-3mois'] = (average_monthly_output * 3).round(3)
    df['Cde-conso'] = np.where(
        df['Stock Phy'] < df['conso-Moy-3mois'],
        df['conso-Moy-3mois'],
        0
    ).round(3)

    # Préparation du tableau des articles à recommander
    reorder_headers = [
        'Code Art.',
        'Libellé 01',
        'Libellé 02',
        'Stock Phy',
        'conso-Moy-3mois',
        'Cde-conso'
    ]

    # On ne garde que les colonnes qui existent réellement dans le fichier
    display_headers = [h for h in reorder_headers if h in df.columns]

    # Filtrage des lignes où une commande est nécessaire
    reorder_df = df[df['Cde-conso'] > 0]

    return df, reorder_df[display_headers]

Writing analysis.py


In [13]:
# @title
%%writefile app.py

import streamlit as st
import pandas as pd
from analysis import process_stock_data

# Configuration de la page
st.set_page_config(layout="wide", page_title="Stock Analysis")

# Titre principal
st.title("📊 Outil d'Analyse de Stock et de Réapprovisionnement")
st.write("Téléchargez votre fichier CSV pour analyser les niveaux de stock et identifier les articles à recommander.")

# Section d'instructions dans un expander
with st.expander("ℹ️ Guide d'Utilisation et Structure du Fichier CSV"):

    st.subheader("Valeur Ajoutée")
    st.write("Cet outil automatise l'analyse des stocks pour prévenir les ruptures. Il calcule les besoins futurs sur la base de la consommation passée et génère une liste claire et exportable des articles nécessitant un réapprovisionnement, simplifiant ainsi la gestion des inventaires.")

    st.subheader("Structure du Fichier CSV")
    st.write("- **Séparateur :** Le fichier doit utiliser un point-virgule (`;`) comme séparateur de colonnes.")
    st.write("- **Encodage :** Utilisez l'encodage `ISO-8859-1` (Europe occidentale).")
    st.write("- **Colonnes :** Le fichier doit contenir les 20 colonnes exactes ci-dessous, dans cet ordre précis :")

    column_list_text = """Code Art.
Libellé 01
Libellé 02
Stock Phy
Mois-1-Somme de Entrées
Mois-1-Somme de Sorties
Mois-2-Somme de Entrées
Mois-2-Somme de Sorties
Mois-3-Somme de Entrées
Mois-3-Somme de Sorties
Mois-4-Somme de Entrées
Mois-4-Somme de Sorties
Mois-5-Somme de Entrées
Mois-5-Somme de Sorties
Mois-6-Somme de Entrées
Mois-6-Somme de Sorties
Mois-7-Somme de Entrées
Mois-7-Somme de Sorties
Mois-8-Somme de Entrées
Mois-8-Somme de Sorties"""
    st.code(column_list_text, language='text')

    st.subheader("Méthode de Calcul")
    st.write("- **Consommation Mensuelle Moyenne :** Additionne toutes les colonnes `'Somme de Sorties...'` et divise par le nombre de mois.")
    st.write("- **Besoin pour 3 mois (`conso-Moy-3mois`) :** Multiplie la consommation moyenne par 3.")
    st.write("- **Quantité à recommander (`Cde-conso`) :** Si le `Stock Phy` est inférieur au besoin pour 3 mois, cette valeur est égale au besoin pour 3 mois ; sinon, elle est de 0.")

# Uploader de fichier
uploaded_file = st.file_uploader("Choisissez votre fichier d'extraction CSV", type=['csv'])

# Traitement et affichage des résultats
if uploaded_file is not None:
    try:
        df_initial = pd.read_csv(uploaded_file, sep=';', decimal=',', encoding='ISO-8859-1')
        df_full_analysis, df_to_reorder = process_stock_data(df_initial)

        st.success("Fichier traité avec succès ! Voici les résultats :")
        st.header("Analyse Complète du Stock")
        st.dataframe(df_full_analysis)
        st.header("Articles à Recommander")
        if df_to_reorder.empty:
            st.info("✅ Bonne nouvelle ! Aucun article à recommander pour le moment.")
        else:
            st.dataframe(df_to_reorder)
    except ValueError as ve:
        st.error(f"Erreur dans les données du fichier : {ve}")
    except Exception as e:
        st.error(f"Une erreur inattendue est survenue : {e}")
        st.warning("Veuillez vérifier que le format du fichier est correct.")

Writing app.py


In [14]:
# @title
!pip install streamlit pyngrok -q
from pyngrok import ngrok
import os

# --- VEUILLEZ METTRE VOTRE VRAI TOKEN NGROK ICI ---
AUTHTOKEN = "32FLxM3IH2YTsSXYXtT602CYtTR_32PgVusEahqzovfDdo2PV"

if "METTEZ_VOTRE_VRAI_TOKEN_NGROK_ICI" in AUTHTOKEN:
    print("🛑 Erreur : Veuillez remplacer le placeholder par votre vrai token ngrok.")
else:
    ngrok.set_auth_token(AUTHTOKEN)

    # Nettoyage des anciens tunnels pour éviter les erreurs
    print("Nettoyage des anciens tunnels ngrok...")
    try:
      for tunnel in ngrok.get_tunnels():
          ngrok.disconnect(tunnel.public_url)
    except:
      pass

    # Lancement de l'application en arrière-plan
    os.system("streamlit run app.py &")

    # Connexion à ngrok avec votre domaine personnalisé
    try:
        public_url = ngrok.connect(8501, domain="ngrok.smflaser.com")
        print("🚀 Votre application est prête !")
        print(f"Cliquez sur ce lien pour y accéder : {public_url}")
    except Exception as e:
        print(f"❌ Erreur lors de la connexion à ngrok avec le domaine personnalisé : {e}")

🚀 Votre application de test est prête !
Cliquez sur ce lien pour y accéder : NgrokTunnel: "https://ngrok.smflaser.com" -> "http://localhost:8501"


Arreter ngrok

In [15]:
# @title
from pyngrok import ngrok

# L'URL exacte du tunnel que vous voulez fermer
tunnel_url_to_close = "https://ngrok.smflaser.com"

try:
    # On récupère tous les tunnels actifs
    for tunnel in ngrok.get_tunnels():
        if tunnel.public_url == tunnel_url_to_close:
            print(f"🛑 Shutting down tunnel: {tunnel.public_url}")
            ngrok.disconnect(tunnel.public_url)
            print("Tunnel successfully closed.")
    print("Cleanup finished.")
except Exception as e:
    print(f"Could not shut down tunnel: {e}")

🛑 Shutting down tunnel: https://ngrok.smflaser.com
Tunnel successfully closed.
Cleanup finished.
